In [6]:
! pip install geopandas


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import geopandas as gpd
import pandas as pd


In [8]:
# Read CSV and define specific dtypes for each column
crimes = pd.read_csv(
    "crime_data.csv",
    dtype={
        "DR_NO": "str",
        "TIME OCC": "str",
        "AREA": "str",
        "AREA NAME": "str",
        "Rpt Dist No": "str",
        "Crm Cd": "str",
        "Crm Cd Desc": "str",
        "Mocodes": "str",
        "Vict Age": "Int64",  # Allows NaN while keeping integer type
        "Vict Sex": "str",
        "Vict Descent": "str",
        "Premis Cd": "Int64",
        "Premis Desc": "str",
        "Weapon Used Cd": "str",
        "Weapon Desc": "str",
        "Status": "str",
        "Status Desc": "str",
        "Crm Cd 1": "str",
        "Crm Cd 2": "str",
        "Crm Cd 3": "str",
        "Crm Cd 4": "str",
        "LOCATION": "str",
        "Cross Street": "str",
        "LAT": "float64",
        "LON": "float64"
    },
    parse_dates=["Date Rptd", "DATE OCC"]  # Parse datetime columns
)

# Verify data types
print(crimes.dtypes)


C:\Users\Zolix\AppData\Local\Temp\ipykernel_16184\3521928656.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  crimes = pd.read_csv(
C:\Users\Zolix\AppData\Local\Temp\ipykernel_16184\3521928656.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  crimes = pd.read_csv(


DR_NO                     object
Date Rptd         datetime64[ns]
DATE OCC          datetime64[ns]
TIME OCC                  object
AREA                      object
AREA NAME                 object
Rpt Dist No               object
Part 1-2                   int64
Crm Cd                    object
Crm Cd Desc               object
Mocodes                   object
Vict Age                   Int64
Vict Sex                  object
Vict Descent              object
Premis Cd                  Int64
Premis Desc               object
Weapon Used Cd            object
Weapon Desc               object
Status                    object
Status Desc               object
Crm Cd 1                  object
Crm Cd 2                  object
Crm Cd 3                  object
Crm Cd 4                  object
LOCATION                  object
Cross Street              object
LAT                      float64
LON                      float64
dtype: object


In [9]:
# Fill NaN values for numeric columns (with -1)
numeric_columns = crimes.select_dtypes(include=['float64', 'int64']).columns
crimes[numeric_columns] = crimes[numeric_columns].fillna(-1)

# Fill NaN values for object columns (with empty string)
object_columns = crimes.select_dtypes(include=['object']).columns
crimes[object_columns] = crimes[object_columns].fillna('')

In [10]:
crimes_gdf = gpd.GeoDataFrame(
    crimes,
    geometry=gpd.points_from_xy(crimes.LON, crimes.LAT),
    crs="EPSG:4326"  # WGS84 (common geographic coordinate system)
)
crimes_gdf.head()


,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON,geometry
0,190326475,2020-03-01,2020-03-01,2130,07,Wilshire,0784,1,510,VEHICLE - STOLEN,...,Adult Arrest,510,998,,,1900 S LONGWOOD AV,,34.0375,-118.3506,POINT (-118.3506 34.0375)
1,200106753,2020-02-09,2020-02-08,1800,01,Central,0182,1,330,BURGLARY FROM VEHICLE,...,Invest Cont,330,998,,,1000 S FLOWER ST,,34.0444,-118.2628,POINT (-118.2628 34.0444)
2,200320258,2020-11-11,2020-11-04,1700,03,Southwest,0356,1,480,BIKE - STOLEN,...,Invest Cont,480,,,,1400 W 37TH ST,,34.0210,-118.3002,POINT (-118.3002 34.021)
3,200907217,2023-05-10,2020-03-10,2037,09,Van Nuys,0964,1,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),...,Invest Cont,343,,,,14000 RIVERSIDE DR,,34.1576,-118.4387,POINT (-118.4387 34.1576)
4,200412582,2020-09-09,2020-09-09,0630,04,Hollenbeck,0413,1,510,VEHICLE - STOLEN,...,Invest Cont,510,,,,200 E AVENUE 28,,34.0820,-118.2130,POINT (-118.213 34.082)


In [11]:
neighbourhoods = gpd.read_file("neighborhood_boundaries.geojson")
neighbourhoods.head()

,OBJECTID,name,geometry
0,1,Adams-Normandie,"POLYGON ((-118.30069 34.03731, -118.30388 34.0..."
1,2,Arleta,"POLYGON ((-118.44255 34.26412, -118.4424 34.26..."
2,3,Arlington Heights,"POLYGON ((-118.31862 34.05306, -118.31673 34.0..."
3,4,Atwater Village,"MULTIPOLYGON (((-118.27886 34.15321, -118.2788..."
4,5,Baldwin Hills/Crenshaw,"POLYGON ((-118.36102 34.02532, -118.36026 34.0..."


In [18]:
crimes_with_neighborhoods = gpd.sjoin(crimes_gdf, neighbourhoods, how="left", predicate="within")
print(crimes_with_neighborhoods.head())
print(crimes_with_neighborhoods.info())

       DR_NO  Date Rptd   DATE OCC TIME OCC AREA   AREA NAME Rpt Dist No  \
0  190326475 2020-03-01 2020-03-01     2130   07    Wilshire        0784   
1  200106753 2020-02-09 2020-02-08     1800   01     Central        0182   
2  200320258 2020-11-11 2020-11-04     1700   03   Southwest        0356   
3  200907217 2023-05-10 2020-03-10     2037   09    Van Nuys        0964   
4  200412582 2020-09-09 2020-09-09     0630   04  Hollenbeck        0413   

   Part 1-2 Crm Cd                               Crm Cd Desc  ... Crm Cd 3  \
0         1    510                          VEHICLE - STOLEN  ...            
1         1    330                     BURGLARY FROM VEHICLE  ...            
2         1    480                             BIKE - STOLEN  ...            
3         1    343  SHOPLIFTING-GRAND THEFT ($950.01 & OVER)  ...            
4         1    510                          VEHICLE - STOLEN  ...            

   Crm Cd 4                                  LOCATION Cross Street      LA

In [19]:
# Drop extra columns and rename joined foreign key
crimes_with_neighborhoods.drop(columns=["index_right"], inplace=True) 
crimes_with_neighborhoods.drop(columns=["name"], inplace=True) 
crimes_with_neighborhoods.drop(columns=["geometry"], inplace=True) 

# Rename the 'OBJECTID' column to 'neighbourhoodId'
crimes_with_neighborhoods.rename(columns={"OBJECTID": "neighbourhoodId"}, inplace=True)

# Drop rows where 'neighbourhoodId' is NaN
crimes_with_neighborhoods.dropna(subset=["neighbourhoodId"], inplace=True)

# Convert 'neighbourhoodId' to integer after ensuring there are no NaN values
crimes_with_neighborhoods["neighbourhoodId"] = crimes_with_neighborhoods["neighbourhoodId"].astype(int)

crimes_with_neighborhoods['Date Rptd'] = pd.to_datetime(crimes_with_neighborhoods['Date Rptd'], errors='coerce')

# Filter the data to keep only rows after 2022
crimes_after_2022 = crimes_with_neighborhoods[crimes_with_neighborhoods['Date Rptd'] > '2022-01-01']
crimes_with_neighborhoods = crimes_after_2022


In [14]:
##crimes_with_neighborhoods_filtered.to_csv('filtered_crimes2.csv', index=False)


In [15]:
! pip install sqlalchemy
! pip install pyodbc


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [20]:
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Float, DateTime, ForeignKey

DATABASE_URI = "mssql+pyodbc://localhost/LosAngelesData?driver=ODBC+Driver+17+for+SQL+Server&Trusted_Connection=yes"
engine = create_engine(DATABASE_URI)

# Create metadata object
metadata = MetaData()

crime_data_table = Table(
    "CrimeData",
    metadata,
    Column("id", Integer, primary_key=True, autoincrement=True),  # Primary key column
     *[
        Column(col, 
               String if crimes_with_neighborhoods[col].dtype == 'object' else
               DateTime if pd.api.types.is_datetime64_any_dtype(crimes_with_neighborhoods[col]) else
               Float if crimes_with_neighborhoods[col].dtype == 'float64' else
               Integer)  # Assuming Integer for other numeric types
        for col in crimes_with_neighborhoods.columns
    ]
)

metadata.drop_all(engine, [crime_data_table])  # Clean start (optional)
metadata.create_all(engine, [crime_data_table])

inserted_row_count = 0
try:
    with engine.connect() as conn:
        result = conn.execute(crime_data_table.insert(), crimes_with_neighborhoods.to_dict(orient="records"))
        conn.commit()
        inserted_row_count += result.rowcount
except Exception as e:
    print(f"Error: {e}")

print(f"Crime data inserted {inserted_row_count} successfully!")


Crime data inserted 59 successfully!
